Basketball reference can be used to get individual player stats for each team to avoid having a players stats but no listed team, and nba.com can be use to team stats as a whole. Will get 10 years through the NBA, starting point will be the 2015-16 season to the 2024-25 season.

Basketball reference:
Teams urls can be access from their coference standings
https://www.basketball-reference.com/leagues/NBA_2024.html
Ending url will be in 'th scope' then 'a href'
NBA.com
https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2024-25
Can change seasons by chaning 2024 and 25 to the year the season started and the year it ended. 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

In [2]:
driver = webdriver.Firefox()

In [3]:
url = "https://www.basketball-reference.com/leagues/NBA_2024.html"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [4]:
BasketballReferenceth = soup.find_all('th', attrs = {'data-stat': 'team_name'})

In [5]:
EasternConference =BasketballReferenceth[1:16]

In [6]:
WesternConference = BasketballReferenceth[17:32]

In [7]:
AllNBATeams = EasternConference+WesternConference

In [8]:
EndingURL = []
for team in AllNBATeams:
    a = str(team)
    b = a.split('href="')[1]
    c = b.split('"')[0]
    EndingURL.append(c)
EndingURL

['/teams/BOS/2024.html',
 '/teams/NYK/2024.html',
 '/teams/MIL/2024.html',
 '/teams/CLE/2024.html',
 '/teams/ORL/2024.html',
 '/teams/IND/2024.html',
 '/teams/PHI/2024.html',
 '/teams/MIA/2024.html',
 '/teams/CHI/2024.html',
 '/teams/ATL/2024.html',
 '/teams/BRK/2024.html',
 '/teams/TOR/2024.html',
 '/teams/CHO/2024.html',
 '/teams/WAS/2024.html',
 '/teams/DET/2024.html',
 '/teams/OKC/2024.html',
 '/teams/DEN/2024.html',
 '/teams/MIN/2024.html',
 '/teams/LAC/2024.html',
 '/teams/DAL/2024.html',
 '/teams/PHO/2024.html',
 '/teams/NOP/2024.html',
 '/teams/LAL/2024.html',
 '/teams/SAC/2024.html',
 '/teams/GSW/2024.html',
 '/teams/HOU/2024.html',
 '/teams/UTA/2024.html',
 '/teams/MEM/2024.html',
 '/teams/SAS/2024.html',
 '/teams/POR/2024.html']

In [9]:
BeginningURL = url.split('/lea')[0]

In [10]:
TeamInitialURL = []
for url in EndingURL:
    a = BeginningURL+url
    TeamInitialURL.append(a)

In [11]:
AllTeamURLs = []
n = 16
for url in TeamInitialURL:
    n = 16
    while n <26:
        a = url.split('24')[0]
        newurl = a+str(n)+'.html'
        AllTeamURLs.append(newurl)
        n= n+1

In [12]:
driver.quit() #To ensure memory useage it not too high before performing task below

In [13]:
driver = webdriver.Firefox()
Player_Stats = pd.DataFrame()
y = -1
tab_count = 0
for alpha in AllTeamURLs:
    TeamAbbreviated = str(alpha)[-13:-10]
    Year = str(alpha)[-9:-5]
    driver.get(alpha)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    gamma = soup.find_all('table', id = 'per_game_stats')[0] #The table is the only one that is per_game, selecting the first one is a safety measure
    tab_count = tab_count+1
    for row in gamma.find_all('tr')[1:-1]: #The first item in gamma is not needed 
        n =0
        a = row
        y =y+1
        for wks in a.find_all('td')[:-1]:  #We don't care much if indivudal awards, just their statline 
            if n == 0:
                b = str(wks)
                c = b.split('</a')[0]
                d = c.split('html">')[1]
                Player_Stats.loc[y,n] = d
                n=n+1
            else:
                e = str(wks).split('</td')[0]
                f = e.split('">')[1]
                if 'strong' in f:
                    g = f.split('</')[0]
                    h = g.split('ng>')[1]
                    Player_Stats.loc[y,n] = h
                    n=n+1
                else:
                    Player_Stats.loc[y, n] = f
                    n=n+1
        Player_Stats.loc[y,n] = TeamAbbreviated
        n=n+1
        Player_Stats.loc[y,n] = Year
        n=n+1
    if tab_count >=30:
        driver.quit() #Due to increased memory useage with each tab opened raises risk of making the computer unable to do anything do to memory being capped
        tab_count = 0
        driver = webdriver.Firefox()


driver.quit()

In [14]:
table_titles = gamma.find_all('th')

In [15]:
table_titles_cleaned = [title.text for title in table_titles]
print(table_titles_cleaned) #We have Player to PTS for the data collected, everything else doesn't matter

['Rk', 'Player', 'Age', 'Pos', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Awards', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '']


In [16]:
Team_Year = ['Team', 'Year']
Player_Stats_colnames = table_titles_cleaned[1:29] + Team_Year

In [17]:
Player_Stats.columns = Player_Stats_colnames

In [18]:
pd.set_option('display.max_columns', None)
Player_Stats

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,Year
0,Avery Bradley,25,SG,76,72,33.4,6.0,13.4,.447,1.9,5.4,.361,4.1,8.1,.505,.520,1.3,1.6,.780,0.6,2.3,2.9,2.1,1.5,0.3,1.4,2.2,15.2,BOS,2016
1,Isaiah Thomas,26,PG,82,79,32.2,7.2,16.9,.428,2.0,5.7,.359,5.2,11.2,.462,.488,5.8,6.6,.871,0.6,2.4,3.0,6.2,1.1,0.1,2.7,2.0,22.2,BOS,2016
2,Jae Crowder,25,SF,73,73,31.6,4.9,11.1,.443,1.7,5.0,.336,3.3,6.2,.529,.518,2.7,3.3,.820,1.0,4.2,5.1,1.8,1.7,0.5,1.1,2.7,14.2,BOS,2016
3,Evan Turner,27,SG,81,12,28.0,4.2,9.3,.456,0.2,1.0,.241,4.0,8.3,.482,.469,1.8,2.2,.827,0.6,4.3,4.9,4.4,1.0,0.3,2.1,1.7,10.5,BOS,2016
4,Marcus Smart,21,PG,61,10,27.3,3.0,8.7,.348,1.0,4.0,.253,2.0,4.7,.427,.405,2.1,2.7,.777,1.2,2.9,4.2,3.0,1.5,0.3,1.3,3.0,9.1,BOS,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6144,Duop Reath,28,C,46,0,10.2,1.5,3.6,.422,0.7,2.3,.321,0.8,1.3,.600,.524,0.4,0.5,.909,1.0,1.0,2.0,0.6,0.3,0.3,0.3,0.9,4.2,POR,2025
6145,Taze Moore,26,SG,2,0,10.0,1.0,5.0,.200,0.5,2.0,.250,0.5,3.0,.167,.250,0.5,1.0,.500,2.0,2.0,4.0,0.5,1.0,0.0,1.0,0.0,3.0,POR,2025
6146,Rayan Rupert,20,SG,52,0,8.8,1.1,2.7,.408,0.3,1.1,.271,0.8,1.6,.506,.465,0.4,0.6,.767,0.5,0.8,1.3,0.5,0.3,0.1,0.6,0.7,3.0,POR,2025
6147,Justin Minaya,25,SF,19,0,5.3,0.4,1.1,.381,0.1,0.5,.200,0.3,0.6,.545,.429,0.0,0.1,.000,0.3,0.3,0.5,0.4,0.3,0.1,0.3,0.6,0.9,POR,2025


In [19]:
driver = webdriver.Firefox()

In [20]:
url = "https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2024-25"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [21]:
n=16
NBA_Team_Stats_URL = []
while n < 26:
    newurl = url[:-5]+str(n-1)+'-'+str(n)
    NBA_Team_Stats_URL.append(newurl)
    n=n+1
NBA_Team_Stats_URL

['https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2015-16',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2016-17',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2017-18',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2018-19',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2019-20',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2020-21',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2021-22',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2022-23',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2023-24',
 'https://www.nba.com/stats/teams/traditional?SeasonType=Regular+Season&Season=2024-25']

In [24]:
NBA_Team_Stats = pd.DataFrame()
y=0
for url in NBA_Team_Stats_URL:
    n = 0
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    a = soup.find_all('table')[2]
    for stat in a.find_all('tr')[1:]:
        n=0
        c = stat.find_all('td')
        for element in c:
            f = str(element)
            if 'span' in f:
                g =f.split('</spa')[0]
                h = g.split('span>')[1]
                NBA_Team_Stats.loc[y,n] = h
                n=n+1
            elif '</a>' in f:
                g =f.split('game">')[1]
                h =g.split('</a')[0]
                NBA_Team_Stats.loc[y,n] = h
                n=n+1
            else:
                g =f.split('</td')[0]
                h =g.split('>')[1]
                NBA_Team_Stats.loc[y,n] = h
                n=n+1
        NBA_Team_Stats.loc[y,n] = url[-7:-3]
        y=y+1
    


driver.quit()

In [25]:
table_titles = a.find_all('th')
table_titles_cleaned = [title.text for title in table_titles]
print(table_titles_cleaned)

['\xa0', 'Team', 'GP', 'W', 'L', 'WIN%', 'Min', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', '+/-', 'GP RANK', 'W RANK', 'L RANK', 'WIN% RANK', 'MIN RANK', 'PTS RANK', 'FGM RANK', 'FGA RANK', 'FG% RANK', '3PM RANK', '3PA RANK', '3P% RANK', 'FTM RANK', 'FTA RANK', 'FT% RANK', 'OREB RANK', 'DREB RANK', 'REB RANK', 'AST RANK', 'TOV RANK', 'STL RANK', 'BLK RANK', 'BLKA RANK', 'PF RANK', 'PFD RANK', '+/- RANK']


In [26]:
Year = ['Year']
Team_Rank = ['Team_Rank']
Team_Stats_colnames = Team_Rank + table_titles_cleaned[1:28] + Year
NBA_Team_Stats.columns = Team_Stats_colnames

In [27]:
NBA_Team_Stats

,Team_Rank,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,Year
0,1,Golden State Warriors,82,73,9,.890,48.5,114.9,42.5,87.3,48.7,13.1,31.6,41.6,16.7,21.8,76.3,10.0,36.2,46.2,28.9,15.2,8.4,6.1,4.1,20.7,19.8,10.8,2015
1,2,San Antonio Spurs,82,67,15,.817,48.1,103.5,40.1,82.9,48.4,7.0,18.5,37.5,16.4,20.4,80.3,9.4,34.5,43.9,24.5,13.1,8.3,5.9,3.9,17.5,19.5,10.6,2015
2,3,Cleveland Cavaliers,82,57,25,.695,48.4,104.3,38.7,84.0,46.0,10.7,29.6,36.2,16.3,21.7,74.8,10.6,33.9,44.5,22.7,13.6,6.7,3.9,4.4,20.3,20.6,6.0,2015
3,4,Toronto Raptors,82,56,26,.683,48.2,102.7,36.7,81.3,45.1,8.6,23.4,37.0,20.8,26.7,77.7,10.2,33.2,43.4,18.7,13.1,7.8,5.5,5.4,19.6,22.0,4.5,2015
4,5,Oklahoma City Thunder,82,55,27,.671,48.4,110.2,41.1,86.4,47.6,8.3,23.7,34.9,19.7,25.2,78.2,13.1,35.6,48.6,23.0,15.9,7.4,5.9,4.5,20.6,20.2,7.3,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,26,Philadelphia 76ers,82,24,58,.293,48.3,109.6,39.7,87.4,45.4,12.7,37.2,34.1,17.5,22.5,78.0,10.4,29.4,39.8,23.2,13.6,9.2,4.5,5.1,19.0,18.7,-6.2,2024
296,27,New Orleans Pelicans,82,21,61,.256,48.2,109.8,40.6,89.9,45.2,12.0,34.6,34.7,16.5,21.9,75.4,12.1,31.5,43.6,25.8,14.6,8.5,5.2,5.2,18.3,17.9,-9.4,2024
297,28,Charlotte Hornets,82,19,63,.232,48.2,105.1,38.3,89.1,43.0,13.0,38.3,33.9,15.5,19.8,78.3,12.2,33.0,45.2,24.3,15.5,7.4,4.5,5.3,18.7,18.1,-9.1,2024
298,29,Washington Wizards,82,18,64,.220,48.1,108.0,39.4,89.9,43.9,13.1,39.1,33.5,16.0,20.6,77.8,10.6,33.1,43.7,25.1,15.6,7.6,5.1,5.2,19.7,18.2,-12.4,2024


In [28]:
sorted(NBA_Team_Stats['Team'].unique())

['Atlanta Hawks',
 'Boston Celtics',
 'Brooklyn Nets',
 'Charlotte Hornets',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Dallas Mavericks',
 'Denver Nuggets',
 'Detroit Pistons',
 'Golden State Warriors',
 'Houston Rockets',
 'Indiana Pacers',
 'LA Clippers',
 'Los Angeles Lakers',
 'Memphis Grizzlies',
 'Miami Heat',
 'Milwaukee Bucks',
 'Minnesota Timberwolves',
 'New Orleans Pelicans',
 'New York Knicks',
 'Oklahoma City Thunder',
 'Orlando Magic',
 'Philadelphia 76ers',
 'Phoenix Suns',
 'Portland Trail Blazers',
 'Sacramento Kings',
 'San Antonio Spurs',
 'Toronto Raptors',
 'Utah Jazz',
 'Washington Wizards']

In [29]:
NBA_Team_Stats['Team'] = NBA_Team_Stats['Team'].replace('LA Clippers', 'Los Angeles Clippers')

In [30]:
driver = webdriver.Firefox()
url = "https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations"
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [31]:
a = soup.find_all('table')[0]

In [32]:
TeamFull=[]
TeamAbb=[]
for element in a.find_all('tr')[1:]:
    b = element
    for value in b.find_all('td'):
        c=str(value)
        if 'br' in c:
            d=c.split('<br/>')[0]
            e =d.split('td>')[1]
            TeamAbb.append(e)
        elif 'title' in c:
            d= c.split('</a')[0]
            e = d.split('">')[1]
            TeamFull.append(e)
        else:
            d=c.split('</td')[0]
            e=d.split('td>')[1]
            f=e[0:3]
            TeamAbb.append(f)

driver.quit()

In [33]:
Team_Full_and_Abb = pd.DataFrame(list(zip(TeamAbb, TeamFull)), columns = ['Abb' , 'Full'])
Team_Full_and_Abb

,Abb,Full
0,ATL,Atlanta Hawks
1,BOS,Boston Celtics
2,BKN,Brooklyn Nets
3,CHA,Charlotte Hornets
4,CHI,Chicago Bulls
5,CLE,Cleveland Cavaliers
6,DAL,Dallas Mavericks
7,DEN,Denver Nuggets
8,DET,Detroit Pistons
9,GSW,Golden State Warriors


In [34]:
Player_Stats = pd.merge(Player_Stats, Team_Full_and_Abb, left_on='Team', right_on='Abb', how = 'left')
Player_Stats =Player_Stats.drop(columns='Abb')
Player_Stats

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,Year,Full
0,Avery Bradley,25,SG,76,72,33.4,6.0,13.4,.447,1.9,5.4,.361,4.1,8.1,.505,.520,1.3,1.6,.780,0.6,2.3,2.9,2.1,1.5,0.3,1.4,2.2,15.2,BOS,2016,Boston Celtics
1,Isaiah Thomas,26,PG,82,79,32.2,7.2,16.9,.428,2.0,5.7,.359,5.2,11.2,.462,.488,5.8,6.6,.871,0.6,2.4,3.0,6.2,1.1,0.1,2.7,2.0,22.2,BOS,2016,Boston Celtics
2,Jae Crowder,25,SF,73,73,31.6,4.9,11.1,.443,1.7,5.0,.336,3.3,6.2,.529,.518,2.7,3.3,.820,1.0,4.2,5.1,1.8,1.7,0.5,1.1,2.7,14.2,BOS,2016,Boston Celtics
3,Evan Turner,27,SG,81,12,28.0,4.2,9.3,.456,0.2,1.0,.241,4.0,8.3,.482,.469,1.8,2.2,.827,0.6,4.3,4.9,4.4,1.0,0.3,2.1,1.7,10.5,BOS,2016,Boston Celtics
4,Marcus Smart,21,PG,61,10,27.3,3.0,8.7,.348,1.0,4.0,.253,2.0,4.7,.427,.405,2.1,2.7,.777,1.2,2.9,4.2,3.0,1.5,0.3,1.3,3.0,9.1,BOS,2016,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6144,Duop Reath,28,C,46,0,10.2,1.5,3.6,.422,0.7,2.3,.321,0.8,1.3,.600,.524,0.4,0.5,.909,1.0,1.0,2.0,0.6,0.3,0.3,0.3,0.9,4.2,POR,2025,Portland Trail Blazers
6145,Taze Moore,26,SG,2,0,10.0,1.0,5.0,.200,0.5,2.0,.250,0.5,3.0,.167,.250,0.5,1.0,.500,2.0,2.0,4.0,0.5,1.0,0.0,1.0,0.0,3.0,POR,2025,Portland Trail Blazers
6146,Rayan Rupert,20,SG,52,0,8.8,1.1,2.7,.408,0.3,1.1,.271,0.8,1.6,.506,.465,0.4,0.6,.767,0.5,0.8,1.3,0.5,0.3,0.1,0.6,0.7,3.0,POR,2025,Portland Trail Blazers
6147,Justin Minaya,25,SF,19,0,5.3,0.4,1.1,.381,0.1,0.5,.200,0.3,0.6,.545,.429,0.0,0.1,.000,0.3,0.3,0.5,0.4,0.3,0.1,0.3,0.6,0.9,POR,2025,Portland Trail Blazers


In [40]:
NBA_Team_Stats = pd.merge(NBA_Team_Stats, Team_Full_and_Abb, left_on='Team', right_on='Full', how = 'left')
NBA_Team_Stats =NBA_Team_Stats.drop(columns='Full')
NBA_Team_Stats

,Team_Rank,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,Year,Abb
0,1,Golden State Warriors,82,73,9,.890,48.5,114.9,42.5,87.3,48.7,13.1,31.6,41.6,16.7,21.8,76.3,10.0,36.2,46.2,28.9,15.2,8.4,6.1,4.1,20.7,19.8,10.8,2015,GSW
1,2,San Antonio Spurs,82,67,15,.817,48.1,103.5,40.1,82.9,48.4,7.0,18.5,37.5,16.4,20.4,80.3,9.4,34.5,43.9,24.5,13.1,8.3,5.9,3.9,17.5,19.5,10.6,2015,SAS
2,3,Cleveland Cavaliers,82,57,25,.695,48.4,104.3,38.7,84.0,46.0,10.7,29.6,36.2,16.3,21.7,74.8,10.6,33.9,44.5,22.7,13.6,6.7,3.9,4.4,20.3,20.6,6.0,2015,CLE
3,4,Toronto Raptors,82,56,26,.683,48.2,102.7,36.7,81.3,45.1,8.6,23.4,37.0,20.8,26.7,77.7,10.2,33.2,43.4,18.7,13.1,7.8,5.5,5.4,19.6,22.0,4.5,2015,TOR
4,5,Oklahoma City Thunder,82,55,27,.671,48.4,110.2,41.1,86.4,47.6,8.3,23.7,34.9,19.7,25.2,78.2,13.1,35.6,48.6,23.0,15.9,7.4,5.9,4.5,20.6,20.2,7.3,2015,OKC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,26,Philadelphia 76ers,82,24,58,.293,48.3,109.6,39.7,87.4,45.4,12.7,37.2,34.1,17.5,22.5,78.0,10.4,29.4,39.8,23.2,13.6,9.2,4.5,5.1,19.0,18.7,-6.2,2024,PHI
296,27,New Orleans Pelicans,82,21,61,.256,48.2,109.8,40.6,89.9,45.2,12.0,34.6,34.7,16.5,21.9,75.4,12.1,31.5,43.6,25.8,14.6,8.5,5.2,5.2,18.3,17.9,-9.4,2024,NOP
297,28,Charlotte Hornets,82,19,63,.232,48.2,105.1,38.3,89.1,43.0,13.0,38.3,33.9,15.5,19.8,78.3,12.2,33.0,45.2,24.3,15.5,7.4,4.5,5.3,18.7,18.1,-9.1,2024,CHA
298,29,Washington Wizards,82,18,64,.220,48.1,108.0,39.4,89.9,43.9,13.1,39.1,33.5,16.0,20.6,77.8,10.6,33.1,43.7,25.1,15.6,7.6,5.1,5.2,19.7,18.2,-12.4,2024,WAS


In [41]:
Player_Stats.to_csv('NBA_Player_Stats.csv', index = False)
NBA_Team_Stats.to_csv('NBA_Team_Stats.csv', index = False)

Double checking tables reveals that the years are off set by 1

In [43]:
Player_Stats['Year'] = Player_Stats['Year'] -1
Player_Stats

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [78]:
print(Player_Stats.dtypes)
print(NBA_Team_Stats.dtypes)

Player    object
Age       object
Pos       object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
eFG%      object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
Team      object
Year      object
Full      object
dtype: object
Team_Rank    object
Team         object
GP           object
W            object
L            object
WIN%         object
Min          object
PTS          object
FGM          object
FGA          object
FG%          object
3PM          object
3PA          object
3P%          object
FTM          object
FTA          object
FT%          object
OREB         object
DREB         object
REB          object
AST          object
TOV          object
STL          object

All rows that can be a integer or float needs to be converted, however there are players whos shooting percentages are blank and have '' (blank) instead of 0

In [80]:
Player_Stats['FG%'] = Player_Stats['FG%'].replace('', 0)
Player_Stats['FG%'] = Player_Stats['3P%'].replace('', 0)
Player_Stats['FG%'] = Player_Stats['2P%'].replace('', 0)
Player_Stats['FG%'] = Player_Stats['eFG%'].replace('', 0)
Player_Stats['FG%'] = Player_Stats['FT%'].replace('', 0)

In [81]:
Player_Stats_Corrected = Player_Stats.apply(pd.to_numeric, errors = 'coerce').fillna(Player_Stats)
print(Player_Stats_Corrected.dtypes)

Player     object
Age         int64
Pos        object
G           int64
GS          int64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%        object
2P        float64
2PA       float64
2P%        object
eFG%       object
FT        float64
FTA       float64
FT%        object
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
Team       object
Year        int64
Full       object
dtype: object


In [82]:
NBA_Team_Stats_Corrected = NBA_Team_Stats.apply(pd.to_numeric, errors = 'coerce').fillna(NBA_Team_Stats)
print(NBA_Team_Stats_Corrected.dtypes)

Team_Rank      int64
Team          object
GP             int64
W              int64
L              int64
WIN%         float64
Min          float64
PTS          float64
FGM          float64
FGA          float64
FG%          float64
3PM          float64
3PA          float64
3P%          float64
FTM          float64
FTA          float64
FT%          float64
OREB         float64
DREB         float64
REB          float64
AST          float64
TOV          float64
STL          float64
BLK          float64
BLKA         float64
PF           float64
PFD          float64
+/-          float64
Year           int64
Abb           object
dtype: object


In [84]:
Player_Stats_Corrected

,Player,Age,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Team,Year,Full
0,Avery Bradley,25,SG,76,72,33.4,6.0,13.4,0.780,1.9,5.4,0.361,4.1,8.1,0.505,0.52,1.3,1.6,0.78,0.6,2.3,2.9,2.1,1.5,0.3,1.4,2.2,15.2,BOS,2016,Boston Celtics
1,Isaiah Thomas,26,PG,82,79,32.2,7.2,16.9,0.871,2.0,5.7,0.359,5.2,11.2,0.462,0.488,5.8,6.6,0.871,0.6,2.4,3.0,6.2,1.1,0.1,2.7,2.0,22.2,BOS,2016,Boston Celtics
2,Jae Crowder,25,SF,73,73,31.6,4.9,11.1,0.820,1.7,5.0,0.336,3.3,6.2,0.529,0.518,2.7,3.3,0.82,1.0,4.2,5.1,1.8,1.7,0.5,1.1,2.7,14.2,BOS,2016,Boston Celtics
3,Evan Turner,27,SG,81,12,28.0,4.2,9.3,0.827,0.2,1.0,0.241,4.0,8.3,0.482,0.469,1.8,2.2,0.827,0.6,4.3,4.9,4.4,1.0,0.3,2.1,1.7,10.5,BOS,2016,Boston Celtics
4,Marcus Smart,21,PG,61,10,27.3,3.0,8.7,0.777,1.0,4.0,0.253,2.0,4.7,0.427,0.405,2.1,2.7,0.777,1.2,2.9,4.2,3.0,1.5,0.3,1.3,3.0,9.1,BOS,2016,Boston Celtics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6144,Duop Reath,28,C,46,0,10.2,1.5,3.6,0.909,0.7,2.3,0.321,0.8,1.3,0.6,0.524,0.4,0.5,0.909,1.0,1.0,2.0,0.6,0.3,0.3,0.3,0.9,4.2,POR,2025,Portland Trail Blazers
6145,Taze Moore,26,SG,2,0,10.0,1.0,5.0,0.500,0.5,2.0,0.25,0.5,3.0,0.167,0.25,0.5,1.0,0.5,2.0,2.0,4.0,0.5,1.0,0.0,1.0,0.0,3.0,POR,2025,Portland Trail Blazers
6146,Rayan Rupert,20,SG,52,0,8.8,1.1,2.7,0.767,0.3,1.1,0.271,0.8,1.6,0.506,0.465,0.4,0.6,0.767,0.5,0.8,1.3,0.5,0.3,0.1,0.6,0.7,3.0,POR,2025,Portland Trail Blazers
6147,Justin Minaya,25,SF,19,0,5.3,0.4,1.1,0.000,0.1,0.5,0.2,0.3,0.6,0.545,0.429,0.0,0.1,0.0,0.3,0.3,0.5,0.4,0.3,0.1,0.3,0.6,0.9,POR,2025,Portland Trail Blazers


In [83]:
NBA_Team_Stats_Corrected

,Team_Rank,Team,GP,W,L,WIN%,Min,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,+/-,Year,Abb
0,1,Golden State Warriors,82,73,9,0.890,48.5,114.9,42.5,87.3,48.7,13.1,31.6,41.6,16.7,21.8,76.3,10.0,36.2,46.2,28.9,15.2,8.4,6.1,4.1,20.7,19.8,10.8,2015,GSW
1,2,San Antonio Spurs,82,67,15,0.817,48.1,103.5,40.1,82.9,48.4,7.0,18.5,37.5,16.4,20.4,80.3,9.4,34.5,43.9,24.5,13.1,8.3,5.9,3.9,17.5,19.5,10.6,2015,SAS
2,3,Cleveland Cavaliers,82,57,25,0.695,48.4,104.3,38.7,84.0,46.0,10.7,29.6,36.2,16.3,21.7,74.8,10.6,33.9,44.5,22.7,13.6,6.7,3.9,4.4,20.3,20.6,6.0,2015,CLE
3,4,Toronto Raptors,82,56,26,0.683,48.2,102.7,36.7,81.3,45.1,8.6,23.4,37.0,20.8,26.7,77.7,10.2,33.2,43.4,18.7,13.1,7.8,5.5,5.4,19.6,22.0,4.5,2015,TOR
4,5,Oklahoma City Thunder,82,55,27,0.671,48.4,110.2,41.1,86.4,47.6,8.3,23.7,34.9,19.7,25.2,78.2,13.1,35.6,48.6,23.0,15.9,7.4,5.9,4.5,20.6,20.2,7.3,2015,OKC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,26,Philadelphia 76ers,82,24,58,0.293,48.3,109.6,39.7,87.4,45.4,12.7,37.2,34.1,17.5,22.5,78.0,10.4,29.4,39.8,23.2,13.6,9.2,4.5,5.1,19.0,18.7,-6.2,2024,PHI
296,27,New Orleans Pelicans,82,21,61,0.256,48.2,109.8,40.6,89.9,45.2,12.0,34.6,34.7,16.5,21.9,75.4,12.1,31.5,43.6,25.8,14.6,8.5,5.2,5.2,18.3,17.9,-9.4,2024,NOP
297,28,Charlotte Hornets,82,19,63,0.232,48.2,105.1,38.3,89.1,43.0,13.0,38.3,33.9,15.5,19.8,78.3,12.2,33.0,45.2,24.3,15.5,7.4,4.5,5.3,18.7,18.1,-9.1,2024,CHA
298,29,Washington Wizards,82,18,64,0.220,48.1,108.0,39.4,89.9,43.9,13.1,39.1,33.5,16.0,20.6,77.8,10.6,33.1,43.7,25.1,15.6,7.6,5.1,5.2,19.7,18.2,-12.4,2024,WAS


In [87]:
Player_With_Team_Stats = pd.merge(NBA_Team_Stats_Corrected, Player_Stats_Corrected, left_on ='Abb', right_on = 'Team')
Player_With_Team_Stats = Player_With_Team_Stats.drop('Abb')

KeyError: "['Abb'] not found in axis"

In [98]:
Player_With_Team_Stats = pd.merge(NBA_Team_Stats_Corrected, Player_Stats_Corrected, left_on =['Abb', 'Year'], right_on = ['Team', 'Year'], how = 'inner')
Player_With_Team_Stats = Player_With_Team_Stats

,Team_Rank,Team_x,GP,W,L,WIN%,Min,PTS_x,FGM,FGA_x,FG%_x,3PM,3PA_x,3P%_x,FTM,FTA_x,FT%_x,OREB,DREB,REB,AST_x,TOV_x,STL_x,BLK_x,BLKA,PF_x,PFD,+/-,Year,Abb,Player,Age,Pos,G,GS,MP,FG,FGA_y,FG%_y,3P,3PA_y,3P%_y,2P,2PA,2P%,eFG%,FT,FTA_y,FT%_y,ORB,DRB,TRB,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,Team_y,Full
0,1,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,12.0,31.2,38.3,17.8,22.6,78.8,9.4,35.0,44.4,30.4,14.8,9.6,6.8,3.8,19.3,19.4,11.6,2016,GSW,Draymond Green,25,PF,81,81,34.7,5.0,10.1,0.696,1.2,3.2,0.388,3.7,6.9,0.537,0.551,2.8,4.1,0.696,1.7,7.8,9.5,7.4,1.5,1.4,3.2,3.0,14.0,GSW,Golden State Warriors
1,1,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,12.0,31.2,38.3,17.8,22.6,78.8,9.4,35.0,44.4,30.4,14.8,9.6,6.8,3.8,19.3,19.4,11.6,2016,GSW,Stephen Curry,27,PG,79,79,34.2,10.2,20.2,0.908,5.1,11.2,0.454,5.1,9.0,0.566,0.63,4.6,5.1,0.908,0.9,4.6,5.4,6.7,2.1,0.2,3.3,2.0,30.1,GSW,Golden State Warriors
2,1,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,12.0,31.2,38.3,17.8,22.6,78.8,9.4,35.0,44.4,30.4,14.8,9.6,6.8,3.8,19.3,19.4,11.6,2016,GSW,Klay Thompson,25,SG,80,80,33.3,8.1,17.3,0.873,3.5,8.1,0.425,4.7,9.2,0.51,0.569,2.4,2.8,0.873,0.4,3.4,3.8,2.1,0.8,0.6,1.7,1.9,22.1,GSW,Golden State Warriors
3,1,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,12.0,31.2,38.3,17.8,22.6,78.8,9.4,35.0,44.4,30.4,14.8,9.6,6.8,3.8,19.3,19.4,11.6,2016,GSW,Harrison Barnes,23,SF,66,59,30.9,4.5,9.6,0.761,1.2,3.2,0.383,3.2,6.3,0.508,0.531,1.5,2.0,0.761,1.2,3.8,4.9,1.8,0.6,0.2,0.9,2.1,11.7,GSW,Golden State Warriors
4,1,Golden State Warriors,82,67,15,0.817,48.2,115.9,43.1,87.1,49.5,12.0,31.2,38.3,17.8,22.6,78.8,9.4,35.0,44.4,30.4,14.8,9.6,6.8,3.8,19.3,19.4,11.6,2016,GSW,Andre Iguodala,32,SF,65,1,26.6,2.7,5.7,0.614,0.8,2.4,0.351,1.9,3.3,0.57,0.552,0.8,1.3,0.614,0.8,3.2,4.0,3.4,1.1,0.3,1.2,1.6,7.0,GSW,Golden State Warriors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4935,30,Utah Jazz,82,17,65,0.207,48.3,111.9,40.3,88.7,45.4,14.0,39.8,35.0,17.3,22.4,77.4,12.0,33.4,45.4,25.5,17.2,6.8,4.4,6.4,18.9,19.3,-9.3,2024,UTA,Micah Potter,25,PF,16,0,11.6,1.2,2.5,0.750,0.6,1.3,0.429,0.6,1.2,0.526,0.588,0.4,0.5,0.75,0.6,2.1,2.7,0.4,0.3,0.4,0.3,0.9,3.3,UTA,Utah Jazz
4936,30,Utah Jazz,82,17,65,0.207,48.3,111.9,40.3,88.7,45.4,14.0,39.8,35.0,17.3,22.4,77.4,12.0,33.4,45.4,25.5,17.2,6.8,4.4,6.4,18.9,19.3,-9.3,2024,UTA,Omer Yurtseven,25,C,48,12,11.4,2.1,3.8,0.679,0.1,0.5,0.208,2.0,3.3,0.588,0.552,0.4,0.6,0.679,1.5,2.8,4.3,0.6,0.2,0.4,0.8,1.1,4.6,UTA,Utah Jazz
4937,30,Utah Jazz,82,17,65,0.207,48.3,111.9,40.3,88.7,45.4,14.0,39.8,35.0,17.3,22.4,77.4,12.0,33.4,45.4,25.5,17.2,6.8,4.4,6.4,18.9,19.3,-9.3,2024,UTA,Jason Preston,24,PG,7,0,10.1,0.9,2.7,0.000,0.0,0.4,0.0,0.9,2.3,0.375,0.316,0.0,0.0,,0.9,1.6,2.4,2.3,0.3,0.1,0.3,0.4,1.7,UTA,Utah Jazz
4938,30,Utah Jazz,82,17,65,0.207,48.3,111.9,40.3,88.7,45.4,14.0,39.8,35.0,17.3,22.4,77.4,12.0,33.4,45.4,25.5,17.2,6.8,4.4,6.4,18.9,19.3,-9.3,2024,UTA,Kira Lewis Jr.,22,PG,12,0,9.9,1.5,3.3,0.778,0.2,1.1,0.154,1.3,2.3,0.593,0.475,0.6,0.8,0.778,0.4,0.6,1.0,1.6,0.3,0.1,0.5,0.3,3.8,UTA,Utah Jazz


,Team_Rank,Team_x,GP,W,L,WIN%,Min,PTS_x,FGM,FGA_x,FG%_x,3PM,3PA_x,3P%_x,FTM,FTA_x,FT%_x,OREB,DREB,REB,AST_x,TOV_x,STL_x,BLK_x,BLKA,PF_x,PFD,+/-,Year,Abb,Player,Age,Pos,G,GS,MP,FG,FGA_y,FG%_y,3P,3PA_y,3P%_y,2P,2PA,2P%,eFG%,FT,FTA_y,FT%_y,ORB,DRB,TRB,AST_y,STL_y,BLK_y,TOV_y,PF_y,PTS_y,Team_y,Full
100,5,Los Angeles Clippers,82,51,31,0.622,48.2,108.7,39.5,83.2,47.5,10.3,27.4,37.5,19.3,26.0,74.5,9.0,34.0,43.0,22.5,13.0,7.5,4.2,3.1,19.8,22.4,4.3,2016,LAC,Jeff Ayres,28,C,17,0,6.3,0.7,1.4,1.000,0.0,0.1,0.0,0.7,1.3,0.545,0.522,0.4,0.4,1.0,0.2,1.1,1.3,0.3,0.0,0.2,0.4,1.1,1.8,LAC,Los Angeles Clippers
101,5,Los Angeles Clippers,82,51,31,0.622,48.2,108.7,39.5,83.2,47.5,10.3,27.4,37.5,19.3,26.0,74.5,9.0,34.0,43.0,22.5,13.0,7.5,4.2,3.1,19.8,22.4,4.3,2016,LAC,Branden Dawson,22,PF,6,0,4.8,0.3,0.8,1.000,0.0,0.0,,0.3,0.8,0.4,0.4,0.2,0.2,1.0,0.3,0.3,0.7,0.0,0.0,0.2,0.0,0.3,0.8,LAC,Los Angeles Clippers
102,5,Los Angeles Clippers,82,51,31,0.622,48.2,108.7,39.5,83.2,47.5,10.3,27.4,37.5,19.3,26.0,74.5,9.0,34.0,43.0,22.5,13.0,7.5,4.2,3.1,19.8,22.4,4.3,2016,LAC,Alex Stepheson,28,PF,4,0,3.0,0.3,0.3,0.000,0.0,0.0,,0.3,0.3,1.0,1.0,0.0,0.5,0.0,0.3,0.3,0.5,0.0,0.0,0.5,0.0,1.0,0.5,LAC,Los Angeles Clippers
103,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,Kyle Lowry,29,PG,77,77,37.0,6.6,15.6,0.811,2.8,7.1,0.388,3.9,8.5,0.461,0.516,5.2,6.4,0.811,0.7,4.0,4.7,6.4,2.1,0.4,2.9,2.7,21.2,TOR,Toronto Raptors
104,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,DeMar DeRozan,26,SG,78,78,35.9,7.9,17.7,0.850,0.6,1.8,0.338,7.3,15.9,0.458,0.463,7.1,8.4,0.85,0.8,3.7,4.5,4.0,1.0,0.3,2.2,2.1,23.5,TOR,Toronto Raptors
105,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,DeMarre Carroll,29,SF,26,22,30.2,4.0,10.4,0.600,1.8,4.5,0.39,2.3,5.8,0.388,0.474,1.2,1.9,0.6,1.2,3.5,4.7,1.0,1.7,0.2,1.1,2.4,11.0,TOR,Toronto Raptors
106,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,Jonas Valančiūnas,23,C,60,59,26.0,5.1,8.9,0.761,0.0,0.0,,5.1,8.9,0.565,0.565,2.7,3.6,0.761,3.1,6.1,9.1,0.7,0.4,1.3,1.4,2.6,12.8,TOR,Toronto Raptors
107,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,Cory Joseph,24,SG,80,4,25.6,3.2,7.3,0.764,0.4,1.4,0.273,2.8,5.9,0.478,0.465,1.7,2.2,0.764,0.5,2.1,2.6,3.1,0.8,0.3,1.3,1.6,8.5,TOR,Toronto Raptors
108,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,Patrick Patterson,26,PF,79,0,25.6,2.6,6.2,0.853,1.3,3.7,0.362,1.2,2.5,0.49,0.521,0.4,0.4,0.853,1.0,3.4,4.3,1.2,0.7,0.4,0.8,1.6,6.9,TOR,Toronto Raptors
109,5,Toronto Raptors,82,51,31,0.622,48.2,106.9,39.2,84.4,46.4,8.8,24.3,36.3,19.7,24.7,79.6,10.6,32.6,43.3,18.5,12.7,8.3,4.9,4.8,20.8,20.3,4.2,2016,TOR,Terrence Ross,24,SF,73,7,23.9,3.7,8.6,0.790,1.8,4.6,0.386,1.9,3.9,0.484,0.536,0.7,0.8,0.79,0.3,2.2,2.5,0.8,0.7,0.3,0.6,1.6,9.9,TOR,Toronto Raptors
